## PLX - Phuture Memecoin Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.50e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.8
liveness_threshold = 90 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["meme-token"]

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'base',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                -0.10,
                                100000,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:
methodology_class.get_category_data()
methodology_class.get_all_coin_data()
methodology_class.filter_and_merge_coin_data(df_to_remove=[stablecoins,ctokens,atokens])
methodology_class.token_supply_check()
methodology_class.asset_maturity_check()
methodology_class.assess_liquidity()

methodology_class.category_data


Excluding test-3, pricing data available only for 75 < 90 days
Excluding broccoli-3, pricing data available only for 66 < 90 days
solana not supported
neon-evm not supported
aptos not supported
solana not supported
solana not supported
solana not supported
aixbt on base returned an invalid API response
solana not supported
aixbt recorded no valid response across any platform
solana not supported
mantle not supported
puff-the-dragon on ethereum returned an invalid API response
puff-the-dragon recorded no valid response across any platform
solana not supported
solana not supported
cronos not supported
solana not supported


,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,platforms
id,,,,,,,,,
shiba-inu,SHIB,Shiba Inu,1.370000e-05,8074080415,18,8077562258,5.892504e+14,5.895045e+14,{'ethereum': '0x95ad61b0a150d79219dcf64e1e6cc0...
pepe,PEPE,Pepe,9.240000e-06,3882834004,35,3882834004,4.206900e+14,4.206900e+14,{'ethereum': '0x6982508145454ce325ddbe47a25d4e...
floki,FLOKI,FLOKI,6.962000e-05,673059792,117,696468263,9.663898e+12,1.000000e+13,{'ethereum': '0xcf0c122c6b73ff809c693db761e7ba...
spx6900,SPX,SPX6900,5.604470e-01,521477657,141,521477657,9.309931e+08,9.309931e+08,{'ethereum': '0xe0f63a424a4439cbe457d80e4f4b51...
based-brett,BRETT,Brett,4.849262e-02,478860596,151,478860598,9.909876e+09,9.909876e+09,{'base': '0x532f27101965dd16442e59d40670faf5eb...
cheems-token,CHEEMS,Cheems Token,1.570000e-06,318425915,201,318425915,2.036730e+14,2.036730e+14,{'binance-smart-chain': '0x0df0587216a4a1bb7d5...
ket,KET,Ket,2.382600e-01,238909276,246,238909276,1.000000e+09,1.000000e+09,{'avalanche': '0xffff003a6bad9b743d65804874293...
mog-coin,MOG,Mog Coin,5.852510e-07,228192403,260,228192403,3.905675e+14,3.905675e+14,{'ethereum': '0xaaee1a9723aadb7afa2810263653a3...
toshi,TOSHI,Toshi,3.855000e-04,161901374,327,161901374,4.206900e+11,4.206900e+11,{'base': '0xac1bd2486aaf3b5c0fc3fd868558b082a5...


In [4]:
methodology_class.slippage_data.head(50)

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,optimistic-ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain
id,,,,,,,,,,,
pepe,-0.267820,NaN,NaN,-0.052181,0.000647,NaN,NaN,binance-smart-chain,0.000647,-0.052181,ethereum
spx6900,NaN,NaN,-0.050295,NaN,-0.002728,NaN,NaN,base,-0.002728,-0.050295,ethereum
shiba-inu,NaN,NaN,NaN,NaN,-0.005009,NaN,NaN,ethereum,-0.005009,-0.005009,ethereum
mog-coin,NaN,NaN,-0.162723,NaN,-0.005910,NaN,NaN,ethereum,-0.005910,-0.005910,ethereum
banana-for-scale-2,NaN,NaN,NaN,-0.008875,NaN,NaN,NaN,binance-smart-chain,-0.008875,-0.008875,binance-smart-chain
floki,NaN,NaN,NaN,-0.999900,-0.021164,NaN,NaN,ethereum,-0.021164,-0.021164,ethereum
toshi,NaN,NaN,-0.021628,NaN,NaN,NaN,NaN,base,-0.021628,-0.021628,base
cheems-token,NaN,NaN,NaN,-0.024356,NaN,NaN,NaN,binance-smart-chain,-0.024356,-0.024356,binance-smart-chain
based-brett,NaN,NaN,-0.034177,NaN,NaN,NaN,NaN,base,-0.034177,-0.034177,base
